## Finding the closest stations to open spaces, food trucks, farmers markets, and murals

In [13]:
import pandas as pd
import requests
import json
import numpy as np
import time as tm
import geopy.distance

In [14]:
# returns number of stations requested by 'k'
# there's a column called "NUM STATIONS" for each open space that gives the number of stations 
# that are actually within the distance 'd' 

In [15]:
# constants
k = 10 # k-nearest neighbors
d = 0.3 # maximum distance in km

In [16]:
# file locations
open_space_source = 'data/raw/Open_Space.geojson'
stations_source = 'data/raw/current_bluebikes_stations.csv'
food_truck_source = 'data/raw/food_truck_schedule.geojson'
farmers_market_source = 'data/raw/winter_farmers_markets.csv'
murals_source = 'data/raw/murals.csv'

In [17]:
stations = pd.read_csv(stations_source, skiprows=1)

In [18]:
# getting average coordinate of each open space using max/min long/lat
def average_coord (i):
    num_coords = len(data['features'][i]['geometry']["coordinates"][0][0])
    min_lat = 1000
    max_lat = -1000
    min_long = 1000
    max_long = -1000
    for j in range(0, num_coords):
        lat = data['features'][i]['geometry']["coordinates"][0][0][j][0]
        long = data['features'][i]['geometry']["coordinates"][0][0][j][1]
        if lat < min_lat: 
            min_lat = lat
        if lat > max_lat:
            max_lat = lat
        if long < min_long: 
            min_long = long
        if long > max_long:
            max_long = long
    return [(min_lat+max_lat)/2, (min_long+max_long)/2]

## Open spaces

In [19]:
f = open(open_space_source)
data = json.load(f)

In [20]:
open_spaces = pd.DataFrame(columns=['OBJECTID', 'OS_ID', 'PARK_ID', 'Name', 'Avg. Lat', 'Avg. Long'])


In [21]:
for i in range(0, len(data['features'])):
    oid = data['features'][i]['properties']['OBJECTID']
    osid = data['features'][i]['properties']['OS_ID']
    parkid = data['features'][i]['properties']['PARK_ID']
    name = data['features'][i]['properties']['SITE_NAME']
    ll = average_coord(i)
    open_spaces.loc[len(open_spaces.index)] = [oid, osid, parkid, name, ll[1], ll[0]]  


In [22]:
open_spaces

,OBJECTID,OS_ID,PARK_ID,Name,Avg. Lat,Avg. Long
0,1,2000,None,Wilson Park,42.340991,-71.145929
1,2,156,156,James H. Roberts Playground,42.359526,-71.125332
2,3,284,284,Smith Playground,42.365072,-71.131046
3,4,115,115,Fern Square,42.359534,-71.133291
4,5,259,259,Ringer Playground,42.350456,-71.138129
...,...,...,...,...,...,...
529,530,7273,7273,Mahoney Park,42.342008,-71.051382
530,531,2106,2106,Boundary I,42.269348,-71.137990
531,1340,7275,7275,Sprague Pond,42.233786,-71.138006
532,2541,7276,7276,Codman Square,42.290418,-71.071082


In [23]:
open_spaces.to_csv('open_space_coords.csv', index=False)

In [24]:
meta_dists = []

for i in range(len(open_spaces)):
    space = (open_spaces["Avg. Lat"][i], open_spaces["Avg. Long"][i])
    dists = []

    for j in range(len(stations)):
        stat = (stations['Latitude'][j], stations['Longitude'][j])
        distance = geopy.distance.geodesic(space, stat).km
        dists += [(distance, stations['Number'][j])]

    meta_dists += [sorted(dists)[0:k]]

In [29]:
meta_dists

[[(0.48175020260611406, 'D32033'),
  (0.6451252195913203, 'D32054'),
  (0.6501131624599871, 'K32002'),
  (0.7178978731245395, 'B32035'),
  (0.9536097884479399, 'D32028'),
  (0.954297150676388, 'D32034'),
  (0.9737540218919234, 'K32011'),
  (1.4195919155401824, 'A32017'),
  (1.4791050371730843, 'D32029'),
  (1.5439523396166428, 'A32001')],
 [(0.27578874572168094, 'A32040'),
  (0.4690397022807988, 'A32011'),
  (0.6053261879248842, 'A32019'),
  (0.784173571269879, 'A32006'),
  (0.8163720968790098, 'A32012'),
  (0.9475343056644482, 'K32006'),
  (0.9480584050070642, 'A32052'),
  (0.978351147785939, 'M32022'),
  (1.0130923935580687, 'A32002'),
  (1.0482694356956508, 'A32005')],
 [(0.22720176045357812, 'A32019'),
  (0.5522332663512377, 'A32011'),
  (0.5720422564623499, 'A32040'),
  (0.6520696225138722, 'A32052'),
  (0.6778668546118113, 'A32005'),
  (1.0558066040021403, 'A32006'),
  (1.093589872515331, 'M32043'),
  (1.0953796981547252, 'M32016'),
  (1.2360733645642479, 'M32017'),
  (1.25610079

In [25]:
meta_dists_saved = meta_dists

In [26]:
open_spaces_top_stations = pd.DataFrame(columns=['OBJECTID', 'OS_ID', 'PARK_ID', 'Name', 'Avg. Lat', 'Avg. Long', 'NUM STATIONS'])


for i in range(k):
    open_spaces_top_stations[("Station {}".format(str(i+1)))] = []
    open_spaces_top_stations[("Station {} Dist ".format(str(i+1)))] = []

In [30]:
for i in range(len(meta_dists)):
    
    
    
    oid = open_spaces['OBJECTID'][i]
    osid = open_spaces['OS_ID'][i]
    parkid = open_spaces['PARK_ID'][i]
    name = open_spaces['Name'][i]
    lat = open_spaces['Avg. Lat'][i]
    long = open_spaces['Avg. Long'][i]
    
    
    to_return = [oid, osid, parkid, name, lat, long] 
    to_return_2 = []
    
    num = 0
    
    for j in range(k):
        stat = meta_dists[i][j][1]
        stat_d = meta_dists[i][j][0]
        
        if stat_d < d:
            num += 1
        
        to_return_2 += [stat, stat_d]
        
    
    to_return_2 = [num] + to_return_2

    to_return += to_return_2
    
    open_spaces_top_stations.loc[len(open_spaces_top_stations.index)] = to_return


In [31]:
open_spaces_top_stations

,OBJECTID,OS_ID,PARK_ID,Name,Avg. Lat,Avg. Long,NUM STATIONS,Station 1,Station 1 Dist,Station 2,...,Station 6,Station 6 Dist,Station 7,Station 7 Dist,Station 8,Station 8 Dist,Station 9,Station 9 Dist,Station 10,Station 10 Dist
0,1,2000,None,Wilson Park,42.340991,-71.145929,0,D32033,0.481750,D32054,...,D32034,0.954297,K32011,0.973754,A32017,1.419592,D32029,1.479105,A32001,1.543952
1,2,156,156,James H. Roberts Playground,42.359526,-71.125332,1,A32040,0.275789,A32011,...,K32006,0.947534,A32052,0.948058,M32022,0.978351,A32002,1.013092,A32005,1.048269
2,3,284,284,Smith Playground,42.365072,-71.131046,1,A32019,0.227202,A32011,...,A32006,1.055807,M32043,1.093590,M32016,1.095380,M32017,1.236073,M32038,1.256101
3,4,115,115,Fern Square,42.359534,-71.133291,0,A32040,0.401155,A32005,...,A32011,0.853399,A32012,1.122541,D32001,1.134382,D32035,1.139529,A32017,1.146023
4,5,259,259,Ringer Playground,42.350456,-71.138129,1,D32034,0.278138,A32001,...,A32045,0.738375,K32004,0.916396,D32028,1.014448,D32001,1.041946,A32012,1.194989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,530,7273,7273,Mahoney Park,42.342008,-71.051382,1,C32024,0.257203,C32029,...,C32016,0.794830,D32018,0.809313,C32035,0.916724,C32091,0.948956,C32025,1.064587
530,531,2106,2106,Boundary I,42.269348,-71.137990,0,C32106,1.553058,E32013,...,C32079,2.034329,C32046,2.049453,C32047,2.142205,C32086,2.181824,C32085,2.277101
531,1340,7275,7275,Sprague Pond,42.233786,-71.138006,0,C32104,2.669196,E32013,...,C32045,5.269233,C32087,5.287377,C32079,5.296514,C32086,5.669837,C32044,5.824563
532,2541,7276,7276,Codman Square,42.290418,-71.071082,1,C32038,0.060420,C32055,...,C32062,0.776719,C32041,1.335041,C32043,1.400501,C23109,1.498176,C32103,1.502327


In [32]:
open_spaces_top_stations.to_csv('open_spaces_nearest_stations_k_{}_d_{}.csv'.format(str(k), str(d)), index=False)

## Food trucks

In [33]:
f = open(food_truck_source)
data = json.load(f)

In [34]:
food_trucks = pd.DataFrame(columns=['OBJECTID', 'Name', 'Lat', 'Long'])

In [35]:
data['features'][0]['properties']['ObjectId']

1

In [36]:
for i in range(len(data['features'])):
    oid = data['features'][i]['properties']['ObjectId']
    name = data['features'][i]['properties']['Truck']
    lat = data['features'][i]['properties']['y']
    long = data['features'][i]['properties']['x']
    food_trucks.loc[len(food_trucks.index)] = [oid, name, lat, long]

In [37]:
food_trucks

,OBJECTID,Name,Lat,Long
0,1,Indian Street Cravings,42.350534,-71.075124
1,2,Moyzilla,42.350534,-71.075124
2,3,Hungry Nomads,42.350534,-71.075124
3,4,Chicken and Rice Guys,42.350534,-71.075124
4,5,Tacos Calleteco,42.350534,-71.075124
5,6,Hungry Nomads,42.350534,-71.075124
6,7,Indian Street Cravings,42.360581,-71.057741
7,8,Chick-fil-A,42.377834,-71.051037
8,9,Chick-fil-A,42.377834,-71.051037
9,10,Indian Street Cravings,42.340478,-71.088974


In [38]:
meta_dists = []

for i in range(len(food_trucks)):
    truck = (food_trucks["Lat"][i], food_trucks["Long"][i])
    dists = []

    for j in range(len(stations)):
        stat = (stations['Latitude'][j], stations['Longitude'][j])
        distance = geopy.distance.geodesic(truck, stat).km
        dists += [(distance, stations['Number'][j])]

    meta_dists += [sorted(dists)[0:k]]

In [41]:
food_trucks_top_stations = pd.DataFrame(columns=['OBJECTID', 'Name', 'Avg. Lat', 'Avg. Long', 'NUM STATIONS'])


for i in range(k):
    food_trucks_top_stations[("Station {}".format(str(i+1)))] = []
    food_trucks_top_stations[("Station {} Dist ".format(str(i+1)))] = []

In [42]:

for i in range(len(meta_dists)):
    
    oid = food_trucks['OBJECTID'][i]
    name = food_trucks['Name'][i]
    lat = food_trucks['Lat'][i]
    long = food_trucks['Long'][i]  
    
    
    
    to_return = [oid, name, lat, long] 
    to_return_2 = []
    
    num = 0
    
    for j in range(k):
        stat = meta_dists[i][j][1]
        stat_d = meta_dists[i][j][0]
        
        if stat_d < d:
            num += 1
        
        to_return_2 += [stat, stat_d]
        
    
    to_return_2 = [num] + to_return_2

    to_return += to_return_2
    
    food_trucks_top_stations.loc[len(food_trucks_top_stations.index)] = to_return


In [43]:
food_trucks_top_stations.to_csv('food_trucks_nearest_stations_k_{}_d_{}.csv'.format(str(k), str(d)), index=False)

In [44]:
food_trucks_top_stations

,OBJECTID,Name,Avg. Lat,Avg. Long,NUM STATIONS,Station 1,Station 1 Dist,Station 2,Station 2 Dist,Station 3,...,Station 6,Station 6 Dist,Station 7,Station 7 Dist,Station 8,Station 8 Dist,Station 9,Station 9 Dist,Station 10,Station 10 Dist
0,1,Indian Street Cravings,42.350534,-71.075124,5,D32013,0.165324,D32005,0.198625,D32045,...,D32046,0.372973,D32007,0.419730,D32017,0.585890,D32042,0.619421,C32008,0.627141
1,2,Moyzilla,42.350534,-71.075124,5,D32013,0.165324,D32005,0.198625,D32045,...,D32046,0.372973,D32007,0.419730,D32017,0.585890,D32042,0.619421,C32008,0.627141
2,3,Hungry Nomads,42.350534,-71.075124,5,D32013,0.165324,D32005,0.198625,D32045,...,D32046,0.372973,D32007,0.419730,D32017,0.585890,D32042,0.619421,C32008,0.627141
3,4,Chicken and Rice Guys,42.350534,-71.075124,5,D32013,0.165324,D32005,0.198625,D32045,...,D32046,0.372973,D32007,0.419730,D32017,0.585890,D32042,0.619421,C32008,0.627141
4,5,Tacos Calleteco,42.350534,-71.075124,5,D32013,0.165324,D32005,0.198625,D32045,...,D32046,0.372973,D32007,0.419730,D32017,0.585890,D32042,0.619421,C32008,0.627141
5,6,Hungry Nomads,42.350534,-71.075124,5,D32013,0.165324,D32005,0.198625,D32045,...,D32046,0.372973,D32007,0.419730,D32017,0.585890,D32042,0.619421,C32008,0.627141
6,7,Indian Street Cravings,42.360581,-71.057741,6,D32009,0.025547,B32008,0.184724,B32032,...,D32010,0.283519,B32061,0.378214,A32053,0.480502,A32013,0.532714,B32056,0.540391
7,8,Chick-fil-A,42.377834,-71.051037,1,D32023,0.182586,D32039,0.516153,D32037,...,A32041,1.011394,D32021,1.011495,D32038,1.103983,D32020,1.144846,A32031,1.327899
8,9,Chick-fil-A,42.377834,-71.051037,1,D32023,0.182586,D32039,0.516153,D32037,...,A32041,1.011394,D32021,1.011495,D32038,1.103983,D32020,1.144846,A32031,1.327899
9,10,Indian Street Cravings,42.340478,-71.088974,1,B32062,0.190111,Z32997,0.372431,B32005,...,C32004,0.607343,D32061,0.618708,C32056,0.641831,B32021,0.681783,B32064,0.705161


## Farmers markets (winter only)

In [45]:
data = pd.read_csv(farmers_market_source)
data

,Market Name,Type,Address,Zip code,Latitude,Longitude,Neighborhood,Market days,Market Hours,First day of Market,Last day of Market,WIC and Senior Farmers Market Nutrition Program Vouchers,Accepts City of Boston Coupons
0,Brighton Farmers Market,In-Person,"640 Washington St, Brighton MA 02135",2135,42.347872,-71.153686,Brighton,"Every other Saturday: 01/27, 02/10, 02/24, 03/...",11am-2pm,01/27/24,03/30/24,WIC,NaN
1,Dorchester Winter Farmers Market,In-Person,"6 Norfolk St., Dorchester MA, 02124",2124,42.289960,-71.071840,Dorchester,Saturday,11am-2pm,01/27/23,03/30/24,"WIC, SFMNPV",NaN
2,East Boston Farmers Market,In-Person,"250 Sumner St, East Boston MA 02128",2128,42.375231,-71.040673,East Boston,Thursday,2pm-6pm,11/16/23,06/20/24,"WIC, SFMNPV",NaN
3,WeGrowMicrogreens,In-Person,"21 Norton St, Hyde Park MA 02136",2136,42.242854,-71.136693,Hyde Park,Friday,1pm-4:30pm,10/27/23,03/08/24,"WIC, SFMNPV",NaN
4,JP Centre St Farmers Market,In-Person,677 Centre St. Bank of America Parking Lot,2130,42.312173,-71.114370,Jamaica Plain,Saturday,12pm-3pm,11/11/24,03/30/24,"WIC, SFMNPV",NaN
5,Roslindale Farmers Market,In-Person,"Adams Park, Roslindale 02131",2131,42.286545,-71.128441,Roslindale,Sunday,11am-2pm,01/7/24,03/31/24,Neither,NaN
6,Dudley Town Common Farmers Market,In-Person,"11 Brook Ave, Roxbury MA 02119",2119,42.325115,-71.074876,Roxbury,Thursday,2:30pm-6:00pm,11/3/23,02/01/24,"WIC, SFMNPV",NaN


In [46]:
meta_dists = []

for i in range(len(data)):
    market = (data["Latitude"][i], data["Longitude"][i])
    dists = []

    for j in range(len(stations)):
        stat = (stations['Latitude'][j], stations['Longitude'][j])
        distance = geopy.distance.geodesic(market, stat).km
        dists += [(distance, stations['Number'][j])]

    meta_dists += [sorted(dists)[0:k]]

In [47]:
farmers_markets_top_stations = pd.DataFrame(columns=['OBJECTID', 'Name', 'Avg. Lat', 'Avg. Long', 'NUM STATIONS'])


for i in range(k):
    farmers_markets_top_stations[("Station {}".format(str(i+1)))] = []
    farmers_markets_top_stations[("Station {} Dist ".format(str(i+1)))] = []

In [48]:
for i in range(len(meta_dists)):
    
    name = data['Market Name'][i]
    lat = data['Latitude'][i]
    long = data['Longitude'][i]
    
    
    to_return = [i, name, lat, long] 
    to_return_2 = []
    
    num = 0
    
    for j in range(k):
        stat = meta_dists[i][j][1]
        stat_d = meta_dists[i][j][0]
        
        if stat_d < d:
            num += 1
        
        to_return_2 += [stat, stat_d]
        
    
    to_return_2 = [num] + to_return_2

    to_return += to_return_2
    
    farmers_markets_top_stations.loc[len(farmers_markets_top_stations.index)] = to_return


In [49]:
farmers_markets_top_stations

,OBJECTID,Name,Avg. Lat,Avg. Long,NUM STATIONS,Station 1,Station 1 Dist,Station 2,Station 2 Dist,Station 3,...,Station 6,Station 6 Dist,Station 7,Station 7 Dist,Station 8,Station 8 Dist,Station 9,Station 9 Dist,Station 10,Station 10 Dist
0,0,Brighton Farmers Market,42.347872,-71.153686,0,D32028,0.305716,D32029,0.559374,D32054,...,D32034,1.131310,D32001,1.196508,A32045,1.382370,B32035,1.392648,N32009,1.417060
1,1,Dorchester Winter Farmers Market,42.289960,-71.071840,2,C32038,0.041527,C32055,0.294878,C32110,...,C32062,0.792420,C32043,1.360089,C32041,1.414260,C32103,1.427107,C23109,1.496906
2,2,East Boston Farmers Market,42.375231,-71.040673,2,A32041,0.124877,A32036,0.215055,A32033,...,A32049,0.665416,A32030,0.713274,D32023,0.762405,A32044,1.027133,A32057,1.061510
3,3,WeGrowMicrogreens,42.242854,-71.136693,0,C32104,1.739842,E32013,1.944959,E32014,...,Z32999,4.423152,C32087,4.433841,C32045,4.512427,C32086,4.749278,C32059,4.831049
4,4,JP Centre St Farmers Market,42.312173,-71.114370,0,E32001,0.357360,E32005,0.605974,D32052,...,E32003,1.128959,E32010,1.249734,E32004,1.354909,K32010,1.500161,C32102,1.500828
5,5,Roslindale Farmers Market,42.286545,-71.128441,2,C32046,0.032757,C32047,0.081500,C32059,...,C32089,1.472931,C32105,1.545995,E32010,1.980068,C32085,2.062637,C32106,2.184864
6,6,Dudley Town Common Farmers Market,42.325115,-71.074876,1,C32015,0.046263,C32022,0.709960,B32017,...,C32005,1.160866,D32059,1.275472,B32055,1.349080,C32061,1.371186,B32023,1.411001


In [51]:
farmers_markets_top_stations.to_csv('farmers_markets_top_stations_k_{}_d_{}.csv'.format(str(k), str(d)), index=False)

## Murals

In [52]:
data = pd.read_csv(murals_source)
data

,Artist,Location,Latitude,Longitude,Title,Year,Description,Neighborhood
0,"Wen-ti Tsen, Yuon Zuo","Oxford St, Boston, MA 02111",42.351697,-71.060571,Chinatown Heritage,1988,The Chinatown Heritage Mural is a copy of a 17...,Chinatown
1,"Vyal ""Vyal One"" Reyes","Underground at Ink Block, 90 Traveler Street, ...",42.345024,-71.060529,Untitled,2017,"Vyal ""Vyal One"" Reyes usualy includes the styl...",South End
2,"Victor ""Marka27"" Quiñonez, Rob ""ProBlak"" Gibbs","Underground at Ink Block, 90 Traveler Street, ...",42.345024,-71.060529,Don’t Let Me Be Misunderstood,2020,"In summer 2020, local artists Rob ""ProBlak"" Gi...",South End
3,"Victor ""Marka27"" Quiñonez, David ""Don Rimx"" Se...","Maurice J Tobin School, 40 Smith St, Roxbury, ...",42.333102,-71.098338,Untitled (Tobin School),2018,One of the many collaborations adorning the To...,Roxbury
4,"Victor ""Marka27"" Quiñonez","Washington Manor, 1701 Washington St. South End",42.337542,-71.076125,Souledad,2022,The City of Boston commissioned artist Victor ...,South End
...,...,...,...,...,...,...,...,...
136,Alex Cook,"Norfolk St. and Ferndale St., Dorchester, MA 0...",42.288290,-71.077099,Beholding the Holy City (Father and Daughter E...,2019,Alex Cook worked with youth from The Boston Pr...,Dorchester
137,Alex Cook,"Centre St. and Starr Lane, Jamaica Plain, MA 0...",42.313140,-71.114120,Starr Lane Landscape,2004,This brightly colored mural depicts several ch...,Jamaica Plain
138,Alex Cook,"Whittier School Apartments, 60 Southern Ave, D...",42.290010,-71.074899,Untitled,2021,Alex worked with youth from The Boston Project...,Dorchester
139,Alex Cook,"618 Shawmut Ave, Roxbury, MA 02118",42.335850,-71.080572,She Sees More Than The World Will Show Her,2009,This mural by Alex Cook features a woman stand...,Roxbury


In [53]:
meta_dists = []

for i in range(len(data)):
    market = (data["Latitude"][i], data["Longitude"][i])
    dists = []

    for j in range(len(stations)):
        stat = (stations['Latitude'][j], stations['Longitude'][j])
        distance = geopy.distance.geodesic(market, stat).km
        dists += [(distance, stations['Number'][j])]

    meta_dists += [sorted(dists)[0:k]]

In [57]:
murals_top_stations = pd.DataFrame(columns=['OBJECTID', 'Title', 'Artist', 'Avg. Lat', 'Avg. Long', 'NUM STATIONS'])


for i in range(k):
    murals_top_stations[("Station {}".format(str(i+1)))] = []
    murals_top_stations[("Station {} Dist ".format(str(i+1)))] = []

In [58]:
murals_top_stations

,OBJECTID,Title,Artist,Avg. Lat,Avg. Long,NUM STATIONS,Station 1,Station 1 Dist,Station 2,Station 2 Dist,...,Station 6,Station 6 Dist,Station 7,Station 7 Dist,Station 8,Station 8 Dist,Station 9,Station 9 Dist,Station 10,Station 10 Dist


In [59]:
for i in range(len(meta_dists)):
    
    title = data['Title'][i]
    artist = data['Artist'][i]
    lat = data['Latitude'][i]
    long = data['Longitude'][i]
    
    
    to_return = [i, title, artist, lat, long] 
    to_return_2 = []
    
    num = 0
    
    for j in range(k):
        stat = meta_dists[i][j][1]
        stat_d = meta_dists[i][j][0]
        
        if stat_d < d:
            num += 1
        
        to_return_2 += [stat, stat_d]
        
    
    to_return_2 = [num] + to_return_2

    to_return += to_return_2
    
    murals_top_stations.loc[len(murals_top_stations.index)] = to_return


In [60]:
murals_top_stations

,OBJECTID,Title,Artist,Avg. Lat,Avg. Long,NUM STATIONS,Station 1,Station 1 Dist,Station 2,Station 2 Dist,...,Station 6,Station 6 Dist,Station 7,Station 7 Dist,Station 8,Station 8 Dist,Station 9,Station 9 Dist,Station 10,Station 10 Dist
0,0,Chinatown Heritage,"Wen-ti Tsen, Yuon Zuo",42.351697,-71.060571,3,D32015,0.106179,D32019,0.190853,...,D32014,0.430442,D32011,0.475075,D32048,0.541910,B32056,0.561224,D32042,0.596729
1,1,Untitled,"Vyal ""Vyal One"" Reyes",42.345024,-71.060529,4,C32035,0.200757,C32025,0.239713,...,D32043,0.505838,D32048,0.571885,C32024,0.660105,D32015,0.709865,C32000,0.749893
2,2,Don’t Let Me Be Misunderstood,"Victor ""Marka27"" Quiñonez, Rob ""ProBlak"" Gibbs",42.345024,-71.060529,4,C32035,0.200757,C32025,0.239713,...,D32043,0.505838,D32048,0.571885,C32024,0.660105,D32015,0.709865,C32000,0.749893
3,3,Untitled (Tobin School),"Victor ""Marka27"" Quiñonez, David ""Don Rimx"" Se...",42.333102,-71.098338,1,B32063,0.240612,C32001,0.336982,...,B32036,0.513104,B32021,0.526368,B32003,0.607191,Z32997,0.762036,B32055,0.786101
4,4,Souledad,"Victor ""Marka27"" Quiñonez",42.337542,-71.076125,1,C32006,0.202858,C32005,0.362363,...,D32061,0.529607,C32093,0.556446,C32060,0.577994,C32004,0.582097,C32061,0.600002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,136,Beholding the Holy City (Father and Daughter E...,Alex Cook,42.288290,-71.077099,0,C32110,0.445928,C32038,0.491996,...,C32103,1.092103,C32062,1.107086,C32092,1.172993,C32066,1.257215,C32094,1.478068
137,137,Starr Lane Landscape,Alex Cook,42.313140,-71.114120,0,E32001,0.466593,E32005,0.627111,...,E32003,1.020895,E32004,1.307737,E32010,1.357138,K32010,1.410189,C32102,1.482043
138,138,Untitled,Alex Cook,42.290010,-71.074899,1,C32038,0.257582,C32110,0.355252,...,C32066,1.034249,C32043,1.127982,C32103,1.327692,C32092,1.399169,C32111,1.491984
139,139,She Sees More Than The World Will Show Her,Alex Cook,42.335850,-71.080572,3,D32059,0.015683,C32005,0.151484,...,C32004,0.560394,B32002,0.612567,B32064,0.613416,C32006,0.614228,C32063,0.653607


In [61]:
murals_top_stations.to_csv('murals_top_stations_k_{}_d_{}.csv'.format(str(k), str(d)), index=False)